### Solutions

#### Question 1

Write a custom class that will be used to model a single bank account.

Your class should implement functionality to:
- allow initialization with values for `first_name`, `last_name`, `account_number`, `balance`, `is_overdraft_allowed`
- keep track of a "ledger" that keeps a record all transactions (just use a list to keep track of this)
    - at a minimum it should keep track of the transaction date (the current UTC datetime) and the amount (positive, or negative to indicate deposits/withdrawals) - later you could add tracking the running balance as well.
- provide read-only properties for `first_name`, `last_name`, `account_number` and `balance`
- provide a property to access the ledger in such a way that a user of this class cannot mutate the ledger directly
- provide a read-write property for `is_overdraft_allowed` that indicates whether overdrafts are allowed on the account.
- provide methods to debit (`def withdraw`) and credit (`def deposit`) transactions that:
    - verify withdrawals against available balance and `is_overdraft_allowed` flag
        - if withdrawal is larger than available balance and overdrafts are not allowed, this should raise a custom `OverdraftNotAllowed` exception.
        - if transaction value is not positive, this should raise a `ValueError` exception (we have separate methods for deposits and withdrawals, and we expect the value to be positive in both cases - one will add to the balance, one will subtract from the balance).
    - add an entry to the ledger with a current UTC timestamp (positive or negative to indicate credit/debit)
    - keeps the available balance updated
- implements a good string representation for the instance (maybe something like `first_name last_name (account_number): balance`

Feel free to expand on the minimum definition I have given here and enhance your custom class.

##### Solution 1

In [1]:
import random
from datetime import datetime, timezone

In [2]:
class BankAccount:
    def __init__(self, *,
                 first_name:str,
                 last_name:str,
                 account_number:int,
                 balance:float= 0,
        is_overdraft_allowed:bool= False,
                overdraft_amount:float= 0):

        # validate: first_name
        if not self.is_pure_str(first_name):
            raise ValueError('first_name must be a string')
        self._first_name = first_name.upper()

        # validate: last_name
        if not self.is_pure_str(last_name):
            raise ValueError('first_name must be a string')
        self._last_name = last_name.upper()

        # validate: account_number
        if isinstance(account_number, float) and not(isinstance(account_number, int)):
            raise ValueError('account_number must be an integer')
        elif not(self.count_digits(account_number, 10)):
            raise ValueError('account_number must be 10 digits')
        else:
            self._account_number = account_number

        # validate: balance
        if not(isinstance(balance, int) or isinstance(balance, float)):
            raise ValueError('balance must be integer or float')
        self._balance = float(balance)

        # validate: is_overdraft_allowed
        if not(isinstance(is_overdraft_allowed, bool)):
            raise ValueError('is_overdraft_allowed must be boolean')
        self.is_overdraft_allowed = is_overdraft_allowed # defaults to False

        # validate: overdraft_amount
        if not(isinstance(balance, int) or isinstance(balance, float)):
            raise ValueError('overdraft_amount must be integer or float')
        if not self.is_overdraft_allowed and overdraft_amount > 0 or overdraft_amount < 0:
            raise ValueError('is_overdraft_allowed is False, therefore setting overdraft amount is not allowed')
        self._overdraft, self._overdraft_amount = [round(overdraft_amount, 2)] * 2

        # initialize ledger
        self._ledger = []

    # validation func
    def is_pure_str(self, value):
        if not isinstance(value, str):
            return False
        try:
            float(value)
            return False
        except ValueError:
            return True
    
    # check if digits is count value. Ex: if count value is 10 and digits is 9, return False
    def count_digits(self, digits, count):
        digits = len([digit for digit in str(digits)])
        return digits == count
    
    # setting read-only properties: first_name, last_name, account_number, balance
    @property
    def first_name(self):
        return self._first_name
    
    @property
    def last_name(self):
        return self._last_name

    @property
    def account_number(self):
        return self._account_number

    @property
    def balance(self):
        return self._balance

    @property
    def ledger(self):
        return self._ledger

    # configure overdraft getter
    @property
    def overdraft(self):
        return self._overdraft

    # contifure overdraft_amount getter
    @property
    def overdraft_amount(self):
        return self._overdraft_amount

    # configure overdraft_amount setter
    @overdraft_amount.setter
    def overdraft_amount(self, value):
        if not(isinstance(value, int) or isinstance(value, float)):
            raise ValueError('overdraft_amount must be integer or float')
        if not self.is_overdraft_allowed:
            raise ValueError('is_overdraft_allowed is False, therefore setting overdraft amount is not allowed')
        if self.overdraft != self.overdraft_amount:
            raise ValueError('BankAccount not qualified for overdraft amount increase')
        self._overdraft, self._overdraft_amount = [round(value, 2)] * 2

    # define ledger_wrap decorator function to add metaprogramming to deposit and withdrawal func
    # this decorator adds every deposit/withdraw transaction to the ledger
    def ledger_wrap(fn):
        def inner(self, amount):
            _dict = {'datetime': datetime.now(timezone.utc), 'transaction_reference': random.randrange(10**13, 10**14)}
            result = fn(self, amount)
            result_copy = result.copy()
            result_copy.update(_dict)
            self.ledger.append(result_copy)
            return result
        return inner
    
    # decorate deposit func with ledger_wrap decorator
    @ledger_wrap
    # deposit func
    def deposit(self, amount):
        # validate amount to be int, float, or greater than zero
        if not(isinstance(amount, int) or isinstance(amount, float)):
            raise ValueError('amount must be integer or float')
        if not(amount > 0):
            raise ValueError('amount must be greater than zero')

        # grab initial balance before transactions
        initial_balance, initial_overdraft = self.balance, self.overdraft

        # deposit + overdraft functionality
        if self.is_overdraft_allowed and self.overdraft < self.overdraft_amount:
            if self.overdraft + amount > self.overdraft_amount:
                remainder = amount - (self.overdraft_amount - self.overdraft)
                self._overdraft = self.overdraft_amount
                self._balance = round(self.balance + remainder, 2)
            else:
                self._overdraft = self.overdraft + amount
            return {
                'credit': amount,
                'initial_balance': initial_balance,
                'current_balance': self.balance,
                'initial_overdraft': initial_overdraft,
                'current_overdraft': self.overdraft
            }
        else:
            self._balance = round(self.balance + amount, 2)
            return {'credit': amount, 'initial_balance': initial_balance, 'current_balance': self.balance}

    # decorate withdraw func with ledger_wrap decorator
    @ledger_wrap
    # withdraw func
    def withdraw(self, amount):
        # validate amount to be int, float, or greater than zero 
        if not(isinstance(amount, int) or isinstance(amount, float)):
            raise ValueError('amount must be integer or float')
        if not(amount > 0):
            raise ValueError('amount must be greater than zero')

        # validate withdrawal, raise custom exception if trying to withdraw beyond overdraft limit
        if amount > self.balance and amount > self.balance + self.overdraft:
            raise self.OverdraftNotAllowed(self.balance + self.overdraft, amount)

        # grab initial balance before transactions
        initial_balance, initial_overdraft = self.balance, self.overdraft

        # withdraw + overdraft functionality
        if amount > self.balance:
            remainder = -(self.balance - amount)
            self._balance = float(0)
            self._overdraft = round(self.overdraft - remainder, 2)
            return {
                'debit': amount,
                'initial_balance': initial_balance,
                'current_balance': self.balance,
                'initial_overdraft': initial_overdraft,
                'current_overdraft': self.overdraft
            }
        else:
            self._balance = round(self.balance - amount, 2)
            return {'debit': amount, 'initial_balance': initial_balance, 'current_balance': self.balance}

    # custom exception 'OverdraftNotAllowed', if withdraw amount exceeds balance
    class OverdraftNotAllowed(Exception):
        def __init__(self, balance, amount):
            self.message = f'Cannot withdraw {amount}. Current Balance: {balance}'
            super().__init__(self.message)

    # class string formatting
    def __str__(self):
        return f'{self.first_name} {self.last_name} ({self.account_number}): {self.balance}'

    # class repr formatting
    def __repr__(self):
        return f'{self.__class__.__name__}(first_name={self.first_name}, last_name={self.last_name}, account_number={self.account_number}, balance={self.balance}, is_overdraft_allowed={self.is_overdraft_allowed}, overdraft={self.overdraft}, overdraft_amount={self.overdraft_amount}, ledger_length={len(self.ledger)})'

    # implement equality
    def __eq__(self, other):
        if not isinstance(other, self.__class__):
            raise ValueError('not comparable')
        return self.account_number == other.account_number

    # implement greater than comparison
    def __gt__(self, other):
        if not isinstance(other, self.__class__):
            raise ValueError('not comparable')
        return self.balance > other.balance

alrightyy...🚀 so in the spirit of taking notes, and for anyone else who is reading this except myself, below I show proof of problems from assignment solved using my `class` code. Enjoy!

so first off, let's create an instance of the `BankAccount` class and assign it to `allwell`

In [3]:
allwell = BankAccount(first_name='Allwell',
                      last_name='Agwu-Okoro',
                      account_number=1234567890,
                      balance=100,
                      #is_overdraft_allowed=True,
                      #overdraft_amount=100
                     )

**1)** allow initialization with values for `first_name`, `last_name`, `account_number`, `balance`, `is_overdraft_allowed` ✅

The `class` takes in all those values. Among them, `balance` defaults to `0.0` and `is_overdraft_allowed` defaults to `False`. Therefore, I don't need to pass them in while I create an instance of the `class`. But since the `balance` property is immutable once created, for the `allwell` instance I passed in a value of `100` for `balance` to start him off with some 💵

In [4]:
allwell.__dict__ # let's take a look at the properties of this BankAccount instance

{'_first_name': 'ALLWELL',
 '_last_name': 'AGWU-OKORO',
 '_account_number': 1234567890,
 '_balance': 100.0,
 'is_overdraft_allowed': False,
 '_overdraft': 0,
 '_overdraft_amount': 0,
 '_ledger': []}

As we can see from the properties of this instance, there are some private properties (properties that start with an underscore `_`), and just one public (I'll come back to that one later).

Also, notice the `_ledger` private property. It's an empty list right now, but this will change once we initiate some transactions. The idea behind the ledger is: it's a transaction tracker (whenever a transaction occurs on a `BankAccount` class instance, the transaction is tracked within this ledger). That's ties in close with the second question (as seen below).

**2)** keep track of a "ledger" that keeps a record all transactions (just use a list to keep track of this) ✅
- at a minimum it should keep track of the transaction date (the current UTC datetime) and the amount (positive, or negative to indicate deposits/withdrawals) - later you could add tracking the running balance as well.

next question!

**3)** provide read-only properties for `first_name`, `last_name`, `account_number` and `balance` ✅

In [5]:
allwell.first_name, allwell.last_name, allwell.account_number, allwell.balance

('ALLWELL', 'AGWU-OKORO', 1234567890, 100.0)

They are all read-only properties now. If I try to write to them or set their values it raises an error. Watch 🍿

In [6]:
allwell.first_name = 'john'
allwell.last_name = 'doe'
allwell.account_number = 69 # 😏
allwell.balance = 100_000_000_000 # USD! very soon God abeg 🤲🏾

AttributeError: can't set attribute

Well, it didn't run the other lines of code because the first line threw an error and exited the code. But you get the point. `first_name`, `last_name`, `account_number`, and `balance` are read-only properties. They can't be written to, which protects and ensures the data integrity of the `BankAccount` class.

**4)** provide a property to access the ledger in such a way that a user of this class cannot mutate the ledger directly ✅

Finally, some *action*. Okay, so here we're gonna see the beauty (the part I enjoyed programming the most) of the ledger tracker. It only happens when a transaction (credit/debit) is made.

Before that, lemme show you something I implemented. The ledger tracker is a list, therefore when there's a transaction, instead of displaying the entire transaction data in the `repr` format display, I instead show the number of transactions tracked. This helps devs know, just from looking at the `repr` format, if a transaction has been performed at all, and if yes, how many.

In [7]:
allwell # in Jupyter, this is the same as doing `repr(allwell)`

BankAccount(first_name=ALLWELL, last_name=AGWU-OKORO, account_number=1234567890, balance=100.0, is_overdraft_allowed=False, overdraft=0, overdraft_amount=0, ledger_length=0)

We can see from the repr that the `ledger_length` is tracking `0` transactions. That's because our `BankAccount` class instance `allwell` has not had any transactions yet. Let's perform some!

While performing transactions, I'd like to share that this action answers some questions from the exercise (such as listed and ticked below):

**5)** provide methods to debit (`def withdraw`) and credit (`def deposit`) transactions that: ✅
1. verify withdrawals against available balance and `is_overdraft_allowed` flag
    1. if withdrawal is larger than available balance and overdrafts are not allowed, this should raise a custom `OverdraftNotAllowed` exception.
    2. if transaction value is not positive, this should raise a `ValueError` exception (we have separate methods for deposits and withdrawals, and we expect the value to be positive in both cases - one will add to the balance, one will subtract from the balance).
2. add an entry to the ledger with a current UTC timestamp (positive or negative to indicate credit/debit) ✅
3. keeps the available balance updated ✅

In [8]:
allwell.balance # let's check allwell's current balance just transactions

100.0

In [9]:
allwell.deposit(100) # a deposit transaction

{'credit': 100, 'initial_balance': 100.0, 'current_balance': 200.0}

After a `deposit` transaction, as well as `withdraw` transaction, the function returns some data about the transaction (as seen above).

Let's perform a `withdraw` transaction.

In [10]:
allwell.withdraw(60) # a withdraw transaction

{'debit': 60, 'initial_balance': 200.0, 'current_balance': 140.0}

Notice the data displayed in return values of `deposit` and `withdraw` functions. They keep track of the `initial_balance` before transaction, as well as credit/debit amount, and `current_balance`.

Now, let's look at the ledger to see if it kept track of both transactions.

First of, let's see if the `ledger_length` corresponds to the number of transactions performed so far. Remember, earlier it was `0`.

In [11]:
allwell

BankAccount(first_name=ALLWELL, last_name=AGWU-OKORO, account_number=1234567890, balance=140.0, is_overdraft_allowed=False, overdraft=0, overdraft_amount=0, ledger_length=2)

`ledger_length` is now `2` showing there have been two transactions performed on this `BankAccount` class instance. If we check the ledger we should see more data on the transactions.

In [13]:
allwell.ledger # ledger tracker

[{'credit': 100,
  'initial_balance': 100.0,
  'current_balance': 200.0,
  'datetime': datetime.datetime(2025, 2, 16, 20, 6, 15, 840871, tzinfo=datetime.timezone.utc),
  'transaction_reference': 50328428576694},
 {'debit': 60,
  'initial_balance': 200.0,
  'current_balance': 140.0,
  'datetime': datetime.datetime(2025, 2, 16, 20, 16, 40, 940729, tzinfo=datetime.timezone.utc),
  'transaction_reference': 76225539946863}]

The `ledger` tracker data (a list of dictionaries (key-value pairs)) contains:
1. transaction debit/credit amount, initial balance, current balance,
2. data about the time of transaction in `datetime.datetime` class type stored (naive) in `UTC`,
3. and a transaction reference (here I implemented a 14-digit `random` number - no one is the same - mirroring a real world transaction reference)

next question!

To answer the rest of the questions in no. **5**-**1** **A** & **B**, let's introduce overdrafts to our `BankAccount` class instance.

Remember, at initialization, `is_overdraft_allowed` defaults to `False`. Meaning, no overdraft. The `allwell` `BankAccount` instance does not have overdraft capability.

Let's imagine that the bank decides to give `allwell` a 50% overdraft amount of `50` due to good financial management.

Before adding overdraft functionality, let's see the current properties of `allwell` instance (particularly overdraft properties).

In [14]:
allwell

BankAccount(first_name=ALLWELL, last_name=AGWU-OKORO, account_number=1234567890, balance=140.0, is_overdraft_allowed=False, overdraft=0, overdraft_amount=0, ledger_length=2)

`is_overdraft_allowed` is `False`, `overdraft` is `0`, and `overdraft_amount` is `0`

Now, let's add overdraft functionality.

In [15]:
allwell.is_overdraft_allowed = True # bank authorizes overdraft functionality for `allwell`
allwell.overdraft_amount = 50 # overdraft amount is `50`
allwell # let's see if the overdraft amount reflects

BankAccount(first_name=ALLWELL, last_name=AGWU-OKORO, account_number=1234567890, balance=140.0, is_overdraft_allowed=True, overdraft=50, overdraft_amount=50, ledger_length=2)

`is_overdraft_allowed` is now `True`, `overdraft` is `50`, and `overdraft_amount` is also `50`.

Now you may be thinking, what is the difference between `overdraft` and `overdraft_amount`? 🤔 (I gotchu 👇🏾)

- `overdraft` is a container (variable) that holds the overdraft (current) amount which operates on `deposit` and `withdraw` transactions.
- While, `overdraft_amount` is a placeholder variable that shows how much overdraft was authorized or awarded to a `BankAccount` class instance.

I hope this explains it succintly.

Let's see some of these features in action. (remember the current `balance` and other property values from the last `repr` display of `allwell`)

In [16]:
allwell.withdraw(150) # withdrawing beyond the balance amount `140`, overdraft will kick in

{'debit': 150,
 'initial_balance': 140.0,
 'current_balance': 0.0,
 'initial_overdraft': 50,
 'current_overdraft': 40.0}

Notice the return value of `withdraw` function is different (showing overdraft data)

If allwell is high as a kite 🪁 and decides to withdraw more money than is available even in his overdraft, he'll receive a custom `OverdraftNotAllowed` exception with a nicely formatted error message. Watch 🍿

In [17]:
allwell.withdraw(50) # exceeding balance and overdraft capacity

OverdraftNotAllowed: Cannot withdraw 50. Current Balance: 40.0

Also, negative `withdraw` or `deposit` values are **not allowed** and would throw an exception.

In [18]:
allwell.withdraw(-10)

ValueError: amount must be greater than zero

In [19]:
allwell.deposit(-10)

ValueError: amount must be greater than zero

That answers questions in no. **5**-**1** **A** & **B** (as seen below):

**5)** provide methods to debit (`def withdraw`) and credit (`def deposit`) transactions that: ✅
1. verify withdrawals against available balance and `is_overdraft_allowed` flag ✅
    1. if withdrawal is larger than available balance and overdrafts are not allowed, this should raise a custom `OverdraftNotAllowed` exception. ✅
    2. if transaction value is not positive, this should raise a `ValueError` exception (we have separate methods for deposits and withdrawals, and we expect the value to be positive in both cases - one will add to the balance, one will subtract from the balance). ✅

Some other cool overdraft features I implemented are:
- while the `overdraft` is depleted, `overdraft_amount` cannot be incremented or decremented. In other words, the bank cannot increase or decrease the `overdraft_amount` if the `overdraft` have been spent. Watch 🍿

In [20]:
# just to recall, `overdraft` is now spent and therefore lesser than `overdraft_amount` assigned to instance
allwell.overdraft, allwell.overdraft_amount

(40.0, 50)

don't get confused, remember the difference between `overdraft` and `overdraft_amount`. See my earlier comments on it if necessary.

In [21]:
# therefore, now, if I (as the bank) try to increase the overdraft amount of this account it'll throw an exception
allwell.overdraft_amount = 100 # increase overdraft from 50 to 100

ValueError: BankAccount not qualified for overdraft amount increase

---

okay... I'm tired now... I'll continue tomorrow. 😴

*...updating...*

---

#### Below is trial code from development (testing out my code and stuff)

---

In [251]:
print(allwell)

ALLWELL AGWU-OKORO (1234567890): 100.0


In [253]:
allwell.__dict__

{'_first_name': 'ALLWELL',
 '_last_name': 'AGWU-OKORO',
 '_account_number': 1234567890,
 '_balance': 100.0,
 'is_overdraft_allowed': False,
 '_overdraft': 0,
 '_overdraft_amount': 0,
 '_ledger': []}

In [255]:
allwell.deposit(10)

{'credit': 10, 'initial_balance': 100.0, 'current_balance': 110.0}

In [256]:
allwell.ledger

[{'credit': 10,
  'initial_balance': 100.0,
  'current_balance': 110.0,
  'datetime': datetime.datetime(2025, 2, 15, 22, 56, 27, 328462, tzinfo=datetime.timezone.utc),
  'transaction_reference': 77870758520333}]

In [257]:
allwell.deposit(350)

{'credit': 350, 'initial_balance': 110.0, 'current_balance': 460.0}

In [258]:
allwell

BankAccount(first_name=ALLWELL, last_name=AGWU-OKORO, account_number=1234567890, balance=460.0, is_overdraft_allowed=False, overdraft=0, overdraft_amount=0, ledger_length=2)

In [259]:
allwell.ledger

[{'credit': 10,
  'initial_balance': 100.0,
  'current_balance': 110.0,
  'datetime': datetime.datetime(2025, 2, 15, 22, 56, 27, 328462, tzinfo=datetime.timezone.utc),
  'transaction_reference': 77870758520333},
 {'credit': 350,
  'initial_balance': 110.0,
  'current_balance': 460.0,
  'datetime': datetime.datetime(2025, 2, 15, 22, 56, 29, 157220, tzinfo=datetime.timezone.utc),
  'transaction_reference': 73141196876612}]

In [260]:
allwell.withdraw(400)

{'debit': 400, 'initial_balance': 460.0, 'current_balance': 60.0}

In [261]:
allwell.ledger

[{'credit': 10,
  'initial_balance': 100.0,
  'current_balance': 110.0,
  'datetime': datetime.datetime(2025, 2, 15, 22, 56, 27, 328462, tzinfo=datetime.timezone.utc),
  'transaction_reference': 77870758520333},
 {'credit': 350,
  'initial_balance': 110.0,
  'current_balance': 460.0,
  'datetime': datetime.datetime(2025, 2, 15, 22, 56, 29, 157220, tzinfo=datetime.timezone.utc),
  'transaction_reference': 73141196876612},
 {'debit': 400,
  'initial_balance': 460.0,
  'current_balance': 60.0,
  'datetime': datetime.datetime(2025, 2, 15, 22, 56, 32, 832258, tzinfo=datetime.timezone.utc),
  'transaction_reference': 95448792844078}]

In [262]:
allwell

BankAccount(first_name=ALLWELL, last_name=AGWU-OKORO, account_number=1234567890, balance=60.0, is_overdraft_allowed=False, overdraft=0, overdraft_amount=0, ledger_length=3)

In [263]:
allwell.is_overdraft_allowed = True
allwell.overdraft_amount = 100
allwell

BankAccount(first_name=ALLWELL, last_name=AGWU-OKORO, account_number=1234567890, balance=60.0, is_overdraft_allowed=True, overdraft=100, overdraft_amount=100, ledger_length=3)

In [264]:
allwell.withdraw(150)

{'debit': 150,
 'initial_balance': 60.0,
 'current_balance': 0.0,
 'initial_overdraft': 100,
 'current_overdraft': 10.0}

In [265]:
allwell.ledger

[{'credit': 10,
  'initial_balance': 100.0,
  'current_balance': 110.0,
  'datetime': datetime.datetime(2025, 2, 15, 22, 56, 27, 328462, tzinfo=datetime.timezone.utc),
  'transaction_reference': 77870758520333},
 {'credit': 350,
  'initial_balance': 110.0,
  'current_balance': 460.0,
  'datetime': datetime.datetime(2025, 2, 15, 22, 56, 29, 157220, tzinfo=datetime.timezone.utc),
  'transaction_reference': 73141196876612},
 {'debit': 400,
  'initial_balance': 460.0,
  'current_balance': 60.0,
  'datetime': datetime.datetime(2025, 2, 15, 22, 56, 32, 832258, tzinfo=datetime.timezone.utc),
  'transaction_reference': 95448792844078},
 {'debit': 150,
  'initial_balance': 60.0,
  'current_balance': 0.0,
  'initial_overdraft': 100,
  'current_overdraft': 10.0,
  'datetime': datetime.datetime(2025, 2, 15, 22, 56, 37, 240645, tzinfo=datetime.timezone.utc),
  'transaction_reference': 32833296440216}]

#### Question 2

Expand on your class above to implement equality (`==`) comparisons between instances of your class.

Two accounts should be considered equal if the account numbers are the same.

##### Solution 2

In [266]:
john = BankAccount(
    first_name='john',
    last_name='cleese',
    account_number=1234567890, # same account number as `allwell`
)

In [267]:
john

BankAccount(first_name=JOHN, last_name=CLEESE, account_number=1234567890, balance=0.0, is_overdraft_allowed=False, overdraft=0, overdraft_amount=0, ledger_length=0)

In [268]:
allwell == john

True

In [269]:
john > allwell

False

In [270]:
allwell > john

False

In [273]:
allwell.balance

0.0

In [274]:
allwell

BankAccount(first_name=ALLWELL, last_name=AGWU-OKORO, account_number=1234567890, balance=0.0, is_overdraft_allowed=True, overdraft=10.0, overdraft_amount=100, ledger_length=4)

In [275]:
allwell.deposit(100)

{'credit': 100,
 'initial_balance': 0.0,
 'current_balance': 10.0,
 'initial_overdraft': 10.0,
 'current_overdraft': 100}

In [277]:
john > allwell

False

In [278]:
allwell > john

True

In [279]:
allwell == john

True